In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler drivertorch:  2.4 ; cuda:  2.4.1
detectron2: 0.6

Copyright (c) 2005-2024 NVIDIA Corporation
Built on Tue_Feb_27_16:28:36_Pacific_Standard_Time_2024
Cuda compilation tools, release 12.4, V12.4.99
Build cuda_12.4.r12.4/compiler.33961263_0


In [2]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer

##
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

In [3]:
def get_hands_dataset_function(directory, class_labels):
    dataset_dicts = []
    json_files = [f for f in os.listdir(directory) if f.endswith(".json")] # JSON FILES
    for idx, json_file in enumerate(json_files):
        json_path = os.path.join(directory, json_file)
        with open(json_path) as f:
            data = json.load(f)

        record = {}
        img_filename = data["imagePath"]
        img_path = os.path.join(directory, img_filename)
        height, width = data["imageHeight"], data["imageWidth"]

        record["file_name"] = img_path
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
    
        objs = []
        for shape in data["shapes"]:
            label = shape["label"]
            if label not in class_labels:
                continue # Skip unknown labels.

            points = np.array(shape["points"], dtype=np.float32)
            px = points[:, 0]
            py = points[:, 1]

            # Create segmentation polygon
            poly = [(x,y) for x, y in zip(px,py)]
            poly = [p for x in poly for p in x] # Flatten list

            obj = {
                "bbox" : [float(np.min(px)), float(np.min(py)),
                        float(np.max(px)), float(np.min(py))],
                "bbox_mode" : BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": class_labels.index(label),
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)

    print(f"Loaded {len(dataset_dicts)} labeled images from {directory}.")
    return dataset_dicts

Registering Datasets

In [ ]:
class_labels = ["0", "1", "2", "3", "4", "5"]

DatasetCatalog.register("hands_train", lambda: get_hands_dataset_function("hand_dataset/train/", class_labels))
MetadataCatalog.get("hands_train").set(thing_classes = class_labels)

DatasetCatalog.register("hands_val", lambda: get_hands_dataset_function("hand_dataset/val/", class_labels))
MetadataCatalog.get("hands_val").set(thing_classes = class_labels)

# Metadata just store static info like class names and color maps.
hands_metadata = MetadataCatalog.get("hands_train") 

Check if all the data is properly loaded.

In [5]:
dataset_dicts = get_hands_dataset_function("hand_dataset/train/", class_labels)
print(f"Loaded {len(dataset_dicts)} images.")
for d in dataset_dicts:
    print(d["file_name"])


Loaded 36 labeled images from hand_dataset/train/.
Loaded 36 images.
hand_dataset/train/0_0.jpg
hand_dataset/train/0_1.jpg
hand_dataset/train/0_2.jpg
hand_dataset/train/0_3.jpg
hand_dataset/train/0_4.jpg
hand_dataset/train/0_5.jpg
hand_dataset/train/1_0.jpg
hand_dataset/train/1_1.jpg
hand_dataset/train/1_2.jpg
hand_dataset/train/1_3.jpg
hand_dataset/train/1_4.jpg
hand_dataset/train/1_5.jpg
hand_dataset/train/2_0.jpg
hand_dataset/train/2_1.jpg
hand_dataset/train/2_2.jpg
hand_dataset/train/2_3.jpg
hand_dataset/train/2_4.jpg
hand_dataset/train/2_5.jpg
hand_dataset/train/3_0.jpg
hand_dataset/train/3_1.jpg
hand_dataset/train/3_2.jpg
hand_dataset/train/3_3.jpg
hand_dataset/train/3_4.jpg
hand_dataset/train/3_5.jpg
hand_dataset/train/4_0.jpg
hand_dataset/train/4_1.jpg
hand_dataset/train/4_2.jpg
hand_dataset/train/4_3.jpg
hand_dataset/train/4_4.jpg
hand_dataset/train/4_5.jpg
hand_dataset/train/5_0.jpg
hand_dataset/train/5_1.jpg
hand_dataset/train/5_2.jpg
hand_dataset/train/5_3.jpg
hand_dataset/

Pull up some sample data for visualization

In [ ]:
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=hands_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2.imshow("Sample Hands",out.get_image()[:, :, ::-1])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Setting up Training Configurations

In [ ]:
from detectron2.engine import DefaultTrainer
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("hands_train",)
cfg.DATASETS.TEST = ("hands_val",)
cfg.DATALOADER.NUM_WORKERS = 2

# Transfer learning from pretrained COCO Instance segmentation model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo

cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 5000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 6  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
cfg.OUTPUT_DIR = "handsydansy_output"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
cfg.MODEL.DEVICE = "cuda"
trainer = DefaultTrainer(cfg) 

Actual Training

In [ ]:
trainer.resume_or_load(resume=False)
trainer.train()
# TRAIN

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

Visualization and Result Analysis

In [ ]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_hands_dataset_function("hand_dataset/val/", class_labels)
for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    print(outputs['instances'])
    v = Visualizer(im[:, :, ::-1],
                   metadata=hands_metadata, 
                   scale=1, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imshow("Hands Validation XDDD",out.get_image()[:, :, ::-1])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

NameError: name 'get' is not defined

In [ ]:
# EVALUATION
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("hands_val", output_dir=os.path.join(cfg.OUTPUT_DIR, "/output_eval"))
val_loader = build_detection_test_loader(cfg, "hands_val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`